In [2]:
#Using smth means: pre-compile and load smth as a package.
using Pkg;
#Set the Gurobi path
ENV["GUROBI_HOME"] = "/Library/gurobi912/macos64/"
#For first timers (comment after): compile Gurobi with the given path
#Pkg.build("Gurobi")
#Add the packages that may not be installed.
Pkg.add(["Latexify", "Polyhedra", "Plots", "Makie", "CDDLib", "JuMP", "Gurobi", "AbstractPlotting", "JLD", "HDF5"])
#Load the packages
using JuMP, Latexify, Gurobi, LinearAlgebra, Polyhedra, CDDLib, Plots, SparseArrays, Makie, AbstractPlotting, JLD, HDF5

display("Initialization done! All set :-)")

    Updating registry at `C:\Users\Alexr\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Manifest.toml`


"Initialization done! All set :-)"

┌ Warning: Everything from AbstractPlotting.jl got moved to Makie.jl. 
│ Have a look at the README for information on how to upgrade: 
│ https://github.com/JuliaPlots/AbstractPlotting.jl
└ @ AbstractPlotting C:\Users\Alexr\.julia\packages\AbstractPlotting\2A5iv\src\AbstractPlotting.jl:261


In [3]:
C=[2.0 4.0 5.0;
   3.0 6.0 8.0; 
   8.0 4.0 9.0]::Matrix{Float64}

3×3 Matrix{Float64}:
 2.0  4.0  5.0
 3.0  6.0  8.0
 8.0  4.0  9.0

In [4]:
data_file_1 = load("../data/X1-3.jld")["X"] 
data_file_2 = load("../data/X2-2.jld")["X"]
data_file_3 = load("../data/X3-2.jld")["X"]
data = data_file_1

25×2 Matrix{Float64}:
 -9.31431     21.8525
 -5.66901     22.8257
 -5.58994     21.7019
 -6.48529     20.4481
 -4.64804    -13.5705
 -6.92318    -13.4546
 -4.92195    -13.169
 -6.18895    -15.1609
 -6.38359    -14.1112
 -4.83052    -13.3698
 -5.54885    -13.7464
 -6.03902    -13.1761
 -9.36696    -13.6319
 -1.01326     -5.99053
  0.0875803   -4.8667
  1.34522     -5.53625
  1.02764     -6.47446
  0.180654    -4.69903
  0.968761    -4.01118
  2.14521     -6.54907
 -3.81989      1.50499
 -4.94545      1.28581
 -3.29144      2.76385
 -2.36855      2.11314
 -4.05179      1.67859

In [5]:
distances = Array{Float64}(undef, size(data,1), size(data,1))
for i = 1:size(data,1)
    for j = 1:size(data,1)
        distances[i,j] = sqrt((data[i,1]-data[j,1])^2 + (data[i,2]-data[j,2])^2)
    end
end

In [6]:
distances

25×25 Matrix{Float64}:
  0.0       3.77298   3.72742   3.15841  …  20.0163   20.9257   20.849
  3.77298   0.0       1.12659   2.51381     20.2023   20.9739   21.2089
  3.72742   1.12659   0.0       1.54065     19.077    19.8519   20.0823
  3.15841   2.51381   1.54065   0.0         17.9704   18.7915   18.9266
 35.729    36.4105   35.2849   34.0682      16.3906   15.8484   15.2607
 35.3879   36.3019   35.1817   33.9055   …  16.6201   16.2203   15.4032
 35.2958   36.0025   34.8773   33.6534      16.0161   15.494    14.8731
 37.1451   37.9901   36.8676   35.6102      18.1574   17.6914   16.9745
 36.0829   36.9438   35.8219   34.5595      17.156    16.7138   15.9611
 35.5065   36.2052   35.0799   33.8584      16.2069   15.6775   15.0685
 35.7975   36.5723   35.4484   34.2074   …  16.6639   16.1753   15.4975
 35.1814   36.0038   34.8809   33.6272      16.1751   15.7237   14.9871
 35.4845   36.6447   35.5351   34.2017      17.4853   17.2304   16.2069
 29.0541   29.1899   28.0681   26.999     

In [7]:
assignmentExample = Model(Gurobi.Optimizer)
@variable(assignmentExample, x[1:size(data,1):noJobs], base_name = "x", lower_bound = 0.0)

LoadError: UndefVarError: noJobs not defined

Academic license - for non-commercial use only
